In [1]:
%pip install numpy matplotlib scipy pennylane qutip qutip-qip

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import scipy as sp
import pennylane as qml
import qutip as qt
from pennylane import qaoa
from pennylane import numpy as np

In [4]:
# Cindi-Ann Jeffery (24695823) & Zander Vermeulen (24700746)
# Trying the example from the paper
b = np.array([0, 1])
A = np.array([[1, -1/3], [-1/3, 1]])

num_wires = 4
dev = qml.device("default.qubit")
ancilla_wire = 'q0_a'
clock_wires = ['q1_c','q2_c']
input_wire = 'q3'
t = 3*np.pi/4

U = np.array([[-0.5 + 0.5j, 0.5 + 0.5j], [0.5 + 0.5j, -0.5 + 0.5j]])
U_sq = ([[0, -1], [-1, 0]])
U_inv = np.array([[-0.5 - 0.5j, 0.5 - 0.5j], [0.5 - 0.5j, -0.5 - 0.5j]])
U_inv_sq = U_sq

@qml.qnode(dev)
def circuit(ancilla_wire, clock_wires, input_wire):
    all_wires = [ancilla_wire, clock_wires[0], clock_wires[1] ,input_wire]
    # Step 1: state preparation
    qml.PauliX(input_wire)
    # Step 2: forward QPE
    qml.QuantumPhaseEstimation(
        U,
        target_wires=input_wire,
        estimation_wires=clock_wires,
    )
    
    # Step 3: timey wimey spacey wacey spinny spin FIXME idk it just feels sus don't you think
    qml.CRY(np.pi*0.2164, wires=[clock_wires[0], ancilla_wire])
    qml.CRY(np.pi, wires=[clock_wires[1], ancilla_wire])
    
    # Step 4: Inverse QPE 
    qml.QFT(wires=clock_wires)
    qml.ControlledQubitUnitary(U_inv_sq, control_wires=clock_wires[1], wires=input_wire)
    qml.ControlledQubitUnitary(U_inv, control_wires=clock_wires[0], wires=input_wire)
    qml.Hadamard(wires=clock_wires[0])
    qml.Hadamard(wires=clock_wires[1])
    
    return qml.probs()
    
# Pretty output
result = circuit(ancilla_wire, clock_wires, input_wire)
print(result)
#column_vector = result.reshape(-1, 1)

#for n in range(15):
#   print(n)
#   print(column_vector[n])
        
# print((result))
# qml.draw_mpl(QuantumPhaseEstimation(unitary,target_wires=input_wire,estimation_wires=clock_wires).decomposition, style="pennylane")()
#qml.draw_mpl(circuit, style="pennylane", show_all_wires=True)(ancilla_wire, clock_wires, input_wire)

[3.87350057e-35 6.25000000e-02 4.63619716e-34 6.25000000e-02
 4.63619716e-34 6.25000000e-02 2.22209113e-01 6.93789233e-03
 1.57947896e-33 6.25000000e-02 4.63619716e-34 6.25000000e-02
 4.63619716e-34 6.25000000e-02 2.22209113e-01 1.73643881e-01]
